# 実践演習 Day 1：streamlitとFastAPIのデモ
このノートブックでは以下の内容を学習します。

- 必要なライブラリのインストールと環境設定
- Hugging Faceからモデルを用いたStreamlitのデモアプリ
- FastAPIとngrokを使用したAPIの公開方法

演習を始める前に、HuggingFaceとngrokのアカウントを作成し、
それぞれのAPIトークンを取得する必要があります。


演習の時間では、以下の3つのディレクトリを順に説明します。

1. 01_streamlit_UI
2. 02_streamlit_app
3. 03_FastAPI

2つ目や3つ目からでも始められる様にノートブックを作成しています。

復習の際にもこのノートブックを役立てていただければと思います。

### 注意事項
「02_streamlit_app」と「03_FastAPI」では、GPUを使用します。

これらを実行する際は、Google Colab画面上のメニューから「編集」→ 「ノートブックの設定」

「ハードウェアアクセラレーター」の項目の中から、「T4 GPU」を選択してください。

このノートブックのデフォルトは「CPU」になっています。

---

# 環境変数の設定（1~3共有）


GitHubから演習用のコードをCloneします。

In [1]:
!git clone https://github.com/matsuolab/lecture-ai-engineering.git

Cloning into 'lecture-ai-engineering'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 41 (delta 7), reused 5 (delta 5), pack-reused 13 (from 1)
Receiving objects: 100% (41/41), 34.04 KiB | 571.00 KiB/s, done.
Resolving deltas: 100% (7/7), done.


必要なAPIトークンを.envに設定します。

「lecture-ai-engineering/day1」の配下に、「.env_template」ファイルが存在しています。

隠しファイルのため表示されていない場合は、画面左側のある、目のアイコンの「隠しファイルの表示」ボタンを押してください。

「.env_template」のファイル名を「.env」に変更します。「.env」ファイルを開くと、以下のような中身になっています。


```
HUGGINGFACE_TOKEN="hf-********"
NGROK_TOKEN="********"
```
ダブルクオーテーションで囲まれた文字列をHuggingfaceのアクセストークンと、ngrokの認証トークンで書き変えてください。

それぞれのアカウントが作成済みであれば、以下のURLからそれぞれのトークンを取得できます。

- Huggingfaceのアクセストークン
https://huggingface.co/docs/hub/security-tokens

- ngrokの認証トークン
https://dashboard.ngrok.com/get-started/your-authtoken

書き換えたら、「.env」ファイルをローカルのPCにダウンロードしてください。

「01_streamlit_UI」から「02_streamlit_app」へ進む際に、CPUからGPUの利用に切り替えるため、セッションが一度切れてしまいます。

その際に、トークンを設定した「.env」ファイルは再作成することになるので、その手間を減らすために「.env」ファイルをダウンロードしておくと良いです。

「.env」ファイルを読み込み、環境変数として設定します。次のセルを実行し、最終的に「True」が表示されていればうまく読み込めています。

In [2]:
!pip install python-dotenv
from dotenv import load_dotenv, find_dotenv

%cd /content/lecture-ai-engineering/day1
load_dotenv(find_dotenv())

/content/lecture-ai-engineering/day1


True

環境変数を一時的に保存しておくコード

In [3]:
import os
import json

def save_env_backup():
        env_backup = {
                "HUGGINGFACE_TOKEN": os.environ.get("HUGGINGFACE_TOKEN"),
                "NGROK_TOKEN": os.environ.get("NGROK_TOKEN")
            }
        with open('/content/env_backup.json', 'w') as f:
              json.dump(env_backup, f)
        print("環境変数のバックアップを作成しました。セッションが切断された場合は restore_env_backup() を実行してください。")

        # 環境変数を復元する関数
        def restore_env_backup():
            try:
                with open('/content/env_backup.json', 'r') as f:
                    env_backup = json.load(f)

                # 環境変数を復元
                for key, value in env_backup.items():
                    os.environ[key] = value

                # .env ファイルを再作成\n",
                with open('/content/lecture-ai-engineering/day1/.env', 'w') as f:
                    for key, value in env_backup.items():
                        f.write(f"{key}={value}")

                print("環境変数を復元しました。")
                return True
            except FileNotFoundError:
                print("バックアップファイルが見つかりません。環境変数を手動で設定してください。")
                return False

        # バックアップを作成
        save_env_backup()

# 01_streamlit_UI

ディレクトリ「01_streamlit_UI」に移動します。

In [4]:
%cd /content/lecture-ai-engineering/day1/01_streamlit_UI

/content/lecture-ai-engineering/day1/01_streamlit_UI


必要なライブラリをインストールします。

In [5]:
%%capture
!pip install -r requirements.txt

# 追加ライブラリのインストール
!pip install streamlit-option-menu streamlit-authenticator

**新しい UI コンポーネントを追加**

In [56]:
%%writefile custom_ui.py
"""
custom_ui.py
-----------------
Streamlit 用のサイドバー＆カスタムページ部品。
• メニュー選択・テーマ切替
• ホームページのカードとプログレスバー
"""
from __future__ import annotations

import time
import streamlit as st
from streamlit_option_menu import option_menu


# ----------------------------------------------------------------------
# サイドバー
# ----------------------------------------------------------------------
_MENU_ITEMS = ["ホーム", "基本要素", "レイアウト", "入力要素", "テーマ設定"]
_ICONS = ["house", "list-task", "columns", "input-cursor", "palette"]


def _apply_dark_theme(enable_dark: bool) -> None:
    """ダークテーマを CSS で適用／解除する。"""
    if enable_dark:
        st.markdown(
            """
            <style>
            .main {background-color: #0E1117; color: #FFFFFF;}
            .sidebar .sidebar-content {background-color: #262730; color: #FFFFFF;}
            </style>
            """,
            unsafe_allow_html=True,
        )


def create_sidebar() -> str:
    """
    カスタムサイドバーを生成し、選択されたメニューを返す。

    Returns
    -------
    str
        現在選択中のページ名。
    """
    with st.sidebar:
        selected = option_menu(
            menu_title="メインメニュー",
            options=_MENU_ITEMS,
            icons=_ICONS,
            menu_icon="cast",
            default_index=0,
        )

        # テーマ切替スイッチ
        if "light_mode" not in st.session_state:
            st.session_state.light_mode = True

        if st.button("🌓 テーマ切替"):
            st.session_state.light_mode = not st.session_state.light_mode

    # サイドバー外でテーマ適用（CSS は 1 度のみ挿入）
    _apply_dark_theme(not st.session_state.light_mode)

    return selected


# ----------------------------------------------------------------------
# 共通ユーティリティ
# ----------------------------------------------------------------------
def page_hourglass() -> None:
    """デモ用のプログレスバーを表示するページ。"""
    st.subheader("改善されたプログレス表示")
    progress_text = "処理中です。しばらくお待ちください..."
    bar = st.progress(0, text=progress_text)
    placeholder = st.empty()

    for percent in range(100):
        time.sleep(0.02)
        bar.progress(percent + 1, text=f"{progress_text} ({percent + 1}%)")
        if percent % 10 == 0:
            placeholder.info(f"Step {percent // 10 + 1}/10 完了")

    bar.empty()
    placeholder.empty()
    st.success("処理が完了しました！")


# ----------------------------------------------------------------------
# ページ切替ハブ
# ----------------------------------------------------------------------
def show_custom_pages(selected: str) -> str:
    """
    選択されたページに対応するコンテンツを描画する。

    Parameters
    ----------
    selected : str
        create_sidebar から返されたページ名。

    Returns
    -------
    str
        同じ値を返すだけ（呼び出し元での再利用用）。
    """
    if selected == "ホーム":
        _render_home()

    # そのほかのページは app.py で処理
    return selected


def _render_home() -> None:
    """ホームページを描画。"""
    st.title("Streamlit UIデモ（改善版）")
    st.write(
        """
        このデモアプリは、Streamlit の基本的な UI 要素を紹介するものです。
        サイドバーから異なるセクションを選択して、さまざまなコンポーネントをお試しください。
        """
    )

    # カード風に 3 カラムで機能案内
    col1, col2, col3 = st.columns(3)
    with col1:
        st.info("**基本要素**\n\nテキスト、ヘッダー、メディアなど")
    with col2:
        st.success("**レイアウト**\n\n列、タブ、エキスパンダーなど")
    with col3:
        st.warning("**入力要素**\n\nボタン、スライダー、テキスト入力など")

    # プログレスバーのデモ起動
    if st.button("プログレスバーデモを表示"):
        page_hourglass()

Overwriting custom_ui.py


ngrokのトークンを使用して、認証を行います。

In [57]:
!ngrok authtoken $$NGROK_TOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


**app.pyの書き換え**

In [58]:
%%writefile app.py
"""Streamlit UI Demo — cleaned & refactored
------------------------------------------------
A single‑file demo app showcasing basic Streamlit components.
Split into small render_* functions for readability.
"""
from __future__ import annotations

import io
from datetime import date as dt_date
from typing import Callable, Dict

import altair as alt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import streamlit as st

from custom_ui import create_sidebar, show_custom_pages

# ----------------------------------------------------------------------------
# ページ共通ユーティリティ
# ----------------------------------------------------------------------------

def random_df(rows: int = 20, cols: int = 3, colnames: list[str] | None = None) -> pd.DataFrame:
    """ランダム DataFrame を生成。"""
    if colnames is None:
        colnames = list("ABC")[:cols]
    return pd.DataFrame(np.random.randn(rows, cols), columns=colnames)


# ----------------------------------------------------------------------------
# 個別ページ描画関数
# ----------------------------------------------------------------------------

def render_basic() -> None:
    """基本 UI 要素ページ"""
    st.title("基本要素")

    # --- テキスト類 ---
    st.header("テキスト要素")
    st.text("これは通常のテキストです。")
    st.markdown("**これはマークダウンテキストです。** *イタリック* や `コード` も使えます。")
    st.info("これは情報メッセージです。")
    st.warning("これは警告メッセージです。")
    st.error("これはエラーメッセージです。")
    st.success("これは成功メッセージです。")

    # --- メディア要素 ---
    st.header("メディア要素")
    tab_chart, tab_df, tab_img = st.tabs(["📈 チャート", "🗃 DataFrame", "🖼 画像"])

    with tab_chart:
        st.subheader("折れ線グラフ")
        chart_data = random_df()
        st.line_chart(chart_data)

        st.subheader("Altair チャート")
        c = (
            alt.Chart(chart_data.reset_index())
            .mark_circle()
            .encode(x="index", y="A", size="B", color="C", tooltip=["A", "B", "C"])
            .interactive()
        )
        st.altair_chart(c, use_container_width=True)

    with tab_df:
        st.subheader("DataFrame の表示")
        df = pd.DataFrame({
            "名前": ["Alice", "Bob", "Charlie", "David"],
            "年齢": [24, 42, 18, 31],
            "都市": ["東京", "大阪", "京都", "名古屋"],
        })
        st.dataframe(df, use_container_width=True)
        st.download_button("CSVとしてダウンロード", df.to_csv(index=False, encoding="utf-8-sig"), "sample_data.csv", "text/csv")

    with tab_img:
        st.subheader("動的に生成した画像")
        fig, ax = plt.subplots()
        x = np.linspace(0, 10, 100)
        ax.plot(x, np.sin(x))
        ax.set_title("サイン波")
        ax.grid(True)
        buf = io.BytesIO()
        fig.savefig(buf, format="png")
        st.image(buf.getvalue(), caption="動的に生成されたサイン波", use_column_width=True)


def render_layout() -> None:
    """レイアウト要素ページ"""
    st.title("レイアウト要素")

    st.header("カラムレイアウト")
    col1, col2, col3 = st.columns(3)
    with col1:
        st.subheader("カラム1")
        st.image("https://via.placeholder.com/150", caption="プレースホルダー画像")
    with col2:
        st.subheader("カラム2")
        st.metric("温度", "28°C", "1.2°C")
    with col3:
        st.subheader("カラム3")
        st.metric("湿度", "65%", "-4%", delta_color="inverse")

    st.header("エキスパンダー")
    with st.expander("詳細を表示"):
        st.write("""
            エキスパンダーを使用すると、長いコンテンツを折りたたむことができます。
            ユーザーが必要なときに展開できるため、画面スペースを節約できます。
        """)
        st.image("https://via.placeholder.com/400x200", caption="大きなプレースホルダー画像")

    st.header("タブ")
    tab1, tab2, tab3 = st.tabs(["Tab 1", "Tab 2", "Tab 3"])
    with tab1:
        st.bar_chart(random_df(10, 3, ["X", "Y", "Z"]))
    with tab2:
        st.line_chart(pd.DataFrame(np.sin(np.linspace(0, 10, 100))))
    with tab3:
        st.area_chart(random_df(10).cumsum())


def render_inputs() -> None:
    """入力要素ページ"""
    st.title("入力要素")

    # --- ボタン ---
    st.header("ボタン")
    if st.button("クリックしてください"):
        st.success("ボタンがクリックされました！")

    # --- チェックボックス ---
    st.header("チェックボックス")
    if st.checkbox("チェックボックスを表示"):
        st.write("チェックボックスがオンになっています。")

    # --- ラジオボタン ---
    st.header("ラジオボタン")
    genre = st.radio("好きな音楽ジャンルは？", ("ロック", "ポップ", "ジャズ", "クラシック"))
    if genre:
        st.write(f"あなたは {genre} を選択しました。")

    # --- セレクトボックス ---
    st.header("セレクトボックス")
    color = st.selectbox("好きな色は？", ("赤", "青", "緑", "黄色"))
    st.write(f"あなたが選んだのは: {color}")

    # --- マルチセレクト ---
    st.header("マルチセレクト")
    fruits = st.multiselect(
        "好きな果物は？",
        ["りんご", "バナナ", "オレンジ", "ぶどう", "いちご"],
        default=["りんご", "バナナ"],
    )
    st.write("あなたが選んだのは: " + ", ".join(fruits))

    # --- スライダー ---
    st.header("スライダー")
    age = st.slider("あなたの年齢は？", 0, 100, 25)
    st.write(f"あなたの年齢: {age}歳")

    # --- 範囲スライダー ---
    values = st.slider("値の範囲を選択:", 0.0, 100.0, (25.0, 75.0))
    st.write(f"選択された範囲: {values[0]} から {values[1]}")

    # --- 日付入力 ---
    st.header("日付入力")
    birth = st.date_input("生年月日を選択してください", dt_date(2000, 1, 1))
    st.write(f"あなたの生年月日: {birth}")

    # --- ファイルアップローダー ---
    st.header("ファイルアップローダー")
    uploaded = st.file_uploader("ファイルを選択してください", type=["csv", "xlsx", "txt", "jpg", "png"])
    if uploaded is not None:
        st.write(f"ファイル名: {uploaded.name} — {uploaded.size} bytes")
        if uploaded.type.startswith("image"):
            st.image(uploaded, caption="アップロードされた画像", use_column_width=True)
        elif uploaded.type == "text/plain":
            string_data = io.StringIO(uploaded.getvalue().decode()).read()
            st.text_area("ファイルの内容", string_data, height=200)
        elif uploaded.type == "text/csv":
            st.dataframe(pd.read_csv(uploaded), use_container_width=True)


# ----------------------------------------------------------------------------
# テーマ設定ページは custom_ui.py 内の CSS で制御
# ----------------------------------------------------------------------------

# ----------------------------------------------------------------------------
# ページ設定 & ルーティング
# ----------------------------------------------------------------------------

def main() -> None:
    st.set_page_config(
        page_title="Streamlit UIデモ（改善版）",
        page_icon="🧊",
        layout="wide",
        initial_sidebar_state="expanded",
    )

    selected = create_sidebar()
    selected = show_custom_pages(selected)  # ホームなど custom_ui 側

    page_table: Dict[str, Callable[[], None]] = {
        "基本要素": render_basic,
        "レイアウト": render_layout,
        "入力要素": render_inputs,
    }

    if selected in page_table:
        page_table[selected]()


if __name__ == "__main__":
    main()

Overwriting app.py


アプリを起動します。

In [ ]:
from pyngrok import ngrok

public_url = ngrok.connect(8501).public_url
print(f"公開URL: {public_url}")
!streamlit run app.py

公開URL: https://f89e-34-125-210-255.ngrok-free.app



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.210.255:8501

/content/lecture-ai-engineering/day1/01_streamlit_UI/app.py:84: UserWarning: Glyph 12469 (\N{KATAKANA LETTER SA}) missing from font(s) DejaVu Sans.
  fig.savefig(buf, format="png")
/content/lecture-ai-engineering/day1/01_streamlit_UI/app.py:84: UserWarning: Glyph 12452 (\N{KATAKANA LETTER I}) missing from font(s) DejaVu Sans.
  fig.savefig(buf, format="png")
/content/lecture-ai-engineering/day1/01_streamlit_UI/app.py:84: UserWarning: Glyph 12531 (\N{KATAKANA LETTER N}) missing from font(s) DejaVu Sans.
  fig.savefig(buf, format="png")
/content/lecture-ai-engineering/day1/01_streamlit_UI/app.py:84: UserWarning: Glyph 27874 (\N{CJK UNIFIED IDEOGRAPH-6CE2}) missing from font(s) DejaVu Sans.
  fig.savefig(buf, format="png")
2025-04-21 05:45:37.838 The `use_column_wid

公開URLの後に記載されているURLにブラウザでアクセスすると、streamlitのUIが表示されます。

app.pyのコメントアウトされている箇所を編集することで、UIがどの様に変化するか確認してみましょう。

streamlitの公式ページには、ギャラリーページがあります。

streamlitを使うとpythonという一つの言語であっても、様々なUIを実現できることがわかると思います。

https://streamlit.io/gallery

後片付けとして、使う必要のないngrokのトンネルを削除します。

In [55]:
from pyngrok import ngrok
ngrok.kill()

# 02_streamlit_app


ディレクトリ「02_streamlit_app」に移動します。

In [22]:
%cd /content/lecture-ai-engineering/day1/02_streamlit_app

/content/lecture-ai-engineering/day1/02_streamlit_app


必要なライブラリをインストールします。

In [23]:
%%capture
!pip install -r requirements.txt

ngrokとhuggigfaceのトークンを使用して、認証を行います。

In [24]:
!ngrok authtoken $$NGROK_TOKEN
!huggingface-cli login --token $$HUGGINGFACE_TOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
The token `AIE_2` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `AIE_2`


stramlitでHuggingfaceのトークン情報を扱うために、streamlit用の設定ファイル（.streamlit）を作成し、トークンの情報を格納します。

In [25]:
# .streamlit/secrets.toml ファイルを作成
import os
import toml

# 設定ファイルのディレクトリ確保
os.makedirs('.streamlit', exist_ok=True)

# 環境変数から取得したトークンを設定ファイルに書き込む
secrets = {
    "huggingface": {
        "token": os.environ.get("HUGGINGFACE_TOKEN", "")
    }
}

# 設定ファイルを書き込む
with open('.streamlit/secrets.toml', 'w') as f:
    toml.dump(secrets, f)

アプリを起動します。

02_streamlit_appでは、Huggingfaceからモデルをダウンロードするため、初回起動には2分程度時間がかかります。

この待ち時間を利用して、app.pyのコードを確認してみましょう。

In [26]:
from pyngrok import ngrok

public_url = ngrok.connect(8501).public_url
print(f"公開URL: {public_url}")
!streamlit run app.py

公開URL: https://a466-34-68-20-49.ngrok-free.app



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.68.20.49:8501

NLTK loaded successfully.
2025-04-20 06:02:45.928364: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745128966.207966    7139 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745128966.283511    7139 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-20 06:02:46.875317: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To 

アプリケーションの機能としては、チャット機能や履歴閲覧があります。

これらの機能を実現するためには、StreamlitによるUI部分だけではなく、SQLiteを使用したチャット履歴の保存やLLMのモデルを呼び出した推論などの処理を組み合わせることで実現しています。

- **`app.py`**: アプリケーションのエントリーポイント。チャット機能、履歴閲覧、サンプルデータ管理のUIを提供します。
- **`ui.py`**: チャットページや履歴閲覧ページなど、アプリケーションのUIロジックを管理します。
- **`llm.py`**: LLMモデルのロードとテキスト生成を行うモジュール。
- **`database.py`**: SQLiteデータベースを使用してチャット履歴やフィードバックを保存・管理します。
- **`metrics.py`**: BLEUスコアやコサイン類似度など、回答の評価指標を計算するモジュール。
- **`data.py`**: サンプルデータの作成やデータベースの初期化を行うモジュール。
- **`config.py`**: アプリケーションの設定（モデル名やデータベースファイル名）を管理します。
- **`requirements.txt`**: このアプリケーションを実行するために必要なPythonパッケージ。

後片付けとして、使う必要のないngrokのトンネルを削除します。

In [27]:
from pyngrok import ngrok
ngrok.kill()

# 03_FastAPI

ディレクトリ「03_FastAPI」に移動します。

In [ ]:
%cd /content/lecture-ai-engineering/day1/03_FastAPI

必要なライブラリをインストールします。

In [ ]:
%%capture
!pip install -r requirements.txt

ngrokとhuggigfaceのトークンを使用して、認証を行います。

In [ ]:
!ngrok authtoken $$NGROK_TOKEN
!huggingface-cli login --token $$HUGGINGFACE_TOKEN

アプリを起動します。

「02_streamlit_app」から続けて「03_FastAPI」を実行している場合は、モデルのダウンロードが済んでいるため、すぐにサービスが立ち上がります。

「03_FastAPI」のみを実行している場合は、初回の起動時にモデルのダウンロードが始まるので、モデルのダウンロードが終わるまで数分間待ちましょう。

In [ ]:
!python app.py

FastAPIが起動すると、APIとクライアントが通信するためのURL（エンドポイント）が作られます。

URLが作られるのと合わせて、Swagger UIというWebインターフェースが作られます。

Swagger UIにアクセスすることで、APIの仕様を確認できたり、APIをテストすることができます。

Swagger UIを利用することで、APIを通してLLMを動かしてみましょう。

後片付けとして、使う必要のないngrokのトンネルを削除します。

In [ ]:
from pyngrok import ngrok
ngrok.kill()